In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
from __future__ import print_function
from numpy import *
from scipy import *
import netCDF4 as nc
import numpy as np
import scipy as sp
import seawater
import datetime as dt

""
from salishsea_tools import (
    nc_tools,
    viz_tools, 
    geo_tools,
    tidetools
)
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc

import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')
import mocsy
import river_201702 as rv
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import arrow
import gsw
import datetime as dt

%matplotlib inline

/home/tjarniko/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
def make_nclen(start,end,ftype, sdir):
    date_ar = []
    sens_ar = []
    doy_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        dddd = tdate.format('DDDD')
        nc_sens = sdir + '/SKOG_1d_*'+ ftype +'*' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        sens_ar.append(tnc_sens[0])
        date_ar.append(ddmmmyy)
        doy_ar.append(dddd)
    return date_ar, sens_ar, doy_ar

In [5]:
start = '2015-01-01'
end = '2015-09-26'

st = dt.datetime(2015,1,1)
en = dt.datetime(2015,9,26)

y_st = st.timetuple().tm_yday
print(y_st)
y_en = en.timetuple().tm_yday
print(y_en)
ts_BR = np.arange(y_st,y_en+1,1)

sdir_preind = '/data/tjarniko/results/PREIND_2nd_2015/PI_2/ncs/'
sdir_br = '/data/tjarniko/results/BR_2nd_2015_cop/SKOG_2/ncs'

dates_preind, files_preind, doy_preind = make_nclen(start,end,'carp', sdir_preind)
dates_br, files_br, doy_br = make_nclen(start,end,'carp', sdir_br)


1
269


In [6]:
for i in range(0,len(doy_br)):
    if i % 10 == 0:
        print(i)
    pi_1 = nc.Dataset(files_preind[i])
    br_1 = nc.Dataset(files_br[i])
    doy = doy_br[i]
    tdate = dates_br[i]

    br_1_co2 = br_1['co2_flux_mmol_m2_s'][0,:,:]
    pi_1_co2 = pi_1['co2_flux_mmol_m2_s'][0,:,:]
    diff_co2 = br_1['co2_flux_mmol_m2_s'][0,:,:] -pi_1['co2_flux_mmol_m2_s'][0,:,:]


    t_cmap = cm.cm.balance
    t_vmin = -0.0008
    t_vmax = 0.0008
    fig = plt.figure(figsize=(12,8))

    ax = fig.add_subplot(1,3,1)
    tplt = np.ma.masked_values(br_1_co2,0)
    mesh = ax.pcolormesh(tplt, cmap=t_cmap, vmin=t_vmin, vmax=t_vmax)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title('baserun')
    cbar = fig.colorbar(mesh, ax=ax)

    ax = fig.add_subplot(1,3,2)
    tplt = np.ma.masked_values(pi_1_co2,0)
    mesh = ax.pcolormesh(tplt, cmap=t_cmap, vmin=t_vmin, vmax=t_vmax)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title('preind')
    cbar = fig.colorbar(mesh, ax=ax)

    ax = fig.add_subplot(1,3,3)
    tplt = np.ma.masked_values(br_1_co2-pi_1_co2,0)
    mesh = ax.pcolormesh(tplt, cmap=t_cmap, vmin=t_vmin, vmax=t_vmax)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title('br-preind')
    cbar = fig.colorbar(mesh, ax=ax)
    fig.suptitle(tdate + ' flux differences, DOY '+ doy, fontsize = 30)
    fname = './FLUX_plot/FLUXDIFF_2NDYR_' + doy +'.png'

    fig.savefig(fname)
    plt.close()
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260


then run bash mov5.sh ./FLUX_plot/ FLUXDIFF_2NDYR_

In [7]:
from salishsea_tools import visualisations as vis
bathy = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')

for i in range(0,len(doy_br)):
    if i % 10 == 0:
        print(i)
    pi_1 = nc.Dataset(files_preind[i])
    br_1 = nc.Dataset(files_br[i])
    doy = doy_br[i]
    tdate = dates_br[i]

    br_1_dic = br_1['dissolved_inorganic_carbon'][0,:,:,:]
    pi_1_dic = pi_1['dissolved_inorganic_carbon'][0,:,:,:]
    diff_dic = br_1['dissolved_inorganic_carbon'][0,:,:,:] -pi_1['dissolved_inorganic_carbon'][0,:,:,:]

    t_cmap = cm.cm.balance
    t_vmin = -50
    t_vmax = 50
    stepsize = 0.5
    fig,ax = plt.subplots(1,1,figsize=(15,5))
    vis.contour_thalweg(ax, diff_dic, bathy, mesh, np.arange(t_vmin, t_vmax, stepsize), cmap = t_cmap)
    ax.set_title(tdate + ' DIC differences, DOY '+ doy, fontsize = 30)
    fname = './FLUX_plot/TWGDIFF_2NDYR_' + doy +'.png'

    fig.savefig(fname)
    plt.close()
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
